In [1]:
import argparse
import time
import sys
import warnings
import logging

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
from joblib import dump, load
import pandas as pd

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
     

In [4]:
!pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install tensorflow_hub



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# python.exe -m pip install --upgrade pip

In [7]:
!pip install tensorflow_text

ERROR: Could not find a version that satisfies the requirement tensorflow_text (from versions: none)
ERROR: No matching distribution found for tensorflow_text

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
#!pip install tensorflow_text
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
#import tensorflow_hub as hub
!pip install tensorflow-gpu
import tensorflow as tf
from tensorflow import keras
#import tensorflow_text
import sklearn.metrics as metrics


ModuleNotFoundError: No module named 'tensorflow_hub'

In [ ]:
!pip install tensorflow_hub


In [ ]:
pip install --upgrade pip


In [161]:
df=pd.read_csv('df.csv')
df.head(5)

,Unnamed: 0,Title,Rating,Link,Image,Size,Occasion,color,Rate
0,0,dresstells women's formal dresses cocktail tea...,50.0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,all,3480
1,1,"dresstells dress with chiffon overlay cape, 20...",1002.0,https://www.amazon.com/DRESSTELLS-Dresses-Wedd...,https://m.media-amazon.com/images/I/61kTmUZqYW...,"s,m,l,xl",any,all,3728
2,2,blencot womens casual short sleeve boho floral...,1011.0,https://www.amazon.com/BLENCOT-Printed-Evening...,https://m.media-amazon.com/images/I/61IVgEOpuB...,"s,m,l,xl",any,all,4060
3,3,hanna nikole women's plus size chiffon ruffle ...,7896.0,https://www.amazon.com/Hanna-Nikole-Chiffon-Po...,https://m.media-amazon.com/images/I/51AKCj7rPk...,"s,m,l,xl",any,all,3811
4,4,dresstells women's formal dresses cocktail tea...,50.0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,all,3480


In [162]:
df.duplicated().sum()

0

In [163]:
df['Title'].apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))

0       dresstells women s formal dresses cocktail tea...
1       dresstells dress with chiffon overlay cape 202...
2       blencot womens casual short sleeve boho floral...
3       hanna nikole women s plus size chiffon ruffle ...
4       dresstells women s formal dresses cocktail tea...
                              ...                        
1445    prinstory men s pajama set sleepwear pjs light...
1446    bulliant mens ratchet belt leather adjustable ...
1447    morwenveo men s linen pants casual long pants ...
1448    hungson mens chinos slim fit stretch flat fron...
1449    long skirts for women maxi boho skirt floral p...
Name: Title, Length: 1450, dtype: object

In [164]:
df.shape

(1450, 9)

In [165]:
df.isna().sum()

Unnamed: 0    0
Title         0
Rating        0
Link          0
Image         0
Size          0
Occasion      0
color         0
Rate          0
dtype: int64

In [166]:
df.reset_index(inplace=True)


In [167]:
df.head(5)

,index,Unnamed: 0,Title,Rating,Link,Image,Size,Occasion,color,Rate
0,0,0,dresstells women's formal dresses cocktail tea...,50.0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,all,3480
1,1,1,"dresstells dress with chiffon overlay cape, 20...",1002.0,https://www.amazon.com/DRESSTELLS-Dresses-Wedd...,https://m.media-amazon.com/images/I/61kTmUZqYW...,"s,m,l,xl",any,all,3728
2,2,2,blencot womens casual short sleeve boho floral...,1011.0,https://www.amazon.com/BLENCOT-Printed-Evening...,https://m.media-amazon.com/images/I/61IVgEOpuB...,"s,m,l,xl",any,all,4060
3,3,3,hanna nikole women's plus size chiffon ruffle ...,7896.0,https://www.amazon.com/Hanna-Nikole-Chiffon-Po...,https://m.media-amazon.com/images/I/51AKCj7rPk...,"s,m,l,xl",any,all,3811
4,4,4,dresstells women's formal dresses cocktail tea...,50.0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,all,3480


In [168]:
df.drop(columns=['Unnamed: 0','index'],axis=1,inplace=True)


In [169]:
df['Rating']=df['Rating'].astype(int)
df.head(3)

,Title,Rating,Link,Image,Size,Occasion,color,Rate
0,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,all,3480
1,"dresstells dress with chiffon overlay cape, 20...",1002,https://www.amazon.com/DRESSTELLS-Dresses-Wedd...,https://m.media-amazon.com/images/I/61kTmUZqYW...,"s,m,l,xl",any,all,3728
2,blencot womens casual short sleeve boho floral...,1011,https://www.amazon.com/BLENCOT-Printed-Evening...,https://m.media-amazon.com/images/I/61IVgEOpuB...,"s,m,l,xl",any,all,4060


In [170]:
df.rename(columns={'Title':'title','Rating':'rating','Link':'link','Image':'image','Size':'size','Occasion':'occasion','Rate':'rate'},inplace=True)

In [171]:
df['occasion'].value_counts()

occasion
any                          448
wedding                      399
birthday                     249
all                          215
wedding,festive occasions     61
christain wedding             46
bday                          32
Name: count, dtype: int64

In [172]:
df.isna().sum()

title       0
rating      0
link        0
image       0
size        0
occasion    0
color       0
rate        0
dtype: int64

In [173]:
df.color.value_counts()

color
black        575
white        295
red          240
different    115
all           76
yellow        57
many          35
mixed         33
golden        24
Name: count, dtype: int64

In [174]:
df['color'].replace({'many':'any','different':'any','all':'any'},inplace=True)

In [175]:
df.color.value_counts()

color
black     575
white     295
red       240
any       226
yellow     57
mixed      33
golden     24
Name: count, dtype: int64

In [176]:
import pandas as pd

# Assuming df is your DataFrame containing the 'title' column
# data = {'title': ['Formal Dress for Wedding', 'Casual Summer Dress', 'Traditional Indian Saree', 'Ethnic Boho Dress']}
# df = pd.DataFrame(data)

# Define a function to categorize dress types
def categorize_title(title):
    title_lower = title.lower()
    if 'a-line' in title_lower or'dress' in title_lower or 'square' in title_lower  or 'western' in title_lower or'midi' in title_lower or'minidress' in title_lower or'party dress' in title_lower or'gown' in title_lower or'maxi' in title_lower or'dokotoo' in title_lower or'square neck' in title_lower or'summer' in title_lower or'ruffle edge sleeves' in title_lower or 'shift' in title_lower or 'wrap' in title_lower or 'maxi' in title_lower or 'midi' in title_lower or 'bodycon' in title_lower or 'sheath' in title_lower or 'shirt' in title_lower or 'off-shoulder' in title_lower or 'peplum' in title_lower:
        return 'Western'
    elif 'wrap' in title_lower or 'shirt' in title_lower or 'off-shoulder' in title_lower or 'peplum' in title_lower or 'indo-western gown' in title_lower:
        return 'Fusion'
    elif ' kaftan' in title_lower or 'thobe' in title_lower or 'Kids' in title_lower or 'Disney' in title_lower or "Kids' Mickey Minnie Mouse"  in title_lower or "Simple Joys by Carter's" in title_lower or 'tunic' in title_lower:
        return 'Children'
    elif  'sari' in title_lower or 'traditioinal' in title_lower or 'saree' in title_lower or 'christian' in title_lower  or 'kurta' in title_lower or 'hindu' in title_lower or 'muslim' in title_lower  or 'indian' in title_lower  or 'chinese' in title_lower or 'african' in title_lower or 'karnataka' in title_lower or 'sarees' in title_lower  or 'lehenga' in title_lower or 'anarkali' in title_lower or 'salwar' in title_lower or 'churidar' in title_lower or 'ghagra' in title_lower or 'patiala' in title_lower:
        return 'Traditional'
    elif ' children' in title_lower or 'onesies' in title_lower or 'Caftans' in title_lower or 'Moroccan' in title_lower or 'dhoti' in title_lower or 'rompers' in title_lower or "children's dresses" in title_lower or "children's" in title_lower :
        return 'Traditional/Ethnic'
    elif 'kimono' in title_lower or 'hanbok' in title_lower or 'ethnic' in title_lower or 'banarasi' in title_lower or 'kufi hat' in title_lower or 'rhinestone' in title_lower or 'sexy' in title_lower  or 'cheongsam' in title_lower or 'club night mini' in title_lower or 'dirndl' in title_lower or 'dashiki' in title_lower or 'sari' in title_lower or 'boubou' in title_lower:
        return 'Ethnic'
    elif 'gown' in title_lower  or 'formal' in title_lower or 'galas' in title_lower  or 'tuxedos' in title_lower or 'gowns' in title_lower or 'wedding' in title_lower or 'red carpet' in title_lower or 'wedding dress (bridal gown)' in title_lower or 'cocktail' in title_lower:
        return 'Formal'
    elif 't-shirt' in title_lower or 'matching' in title_lower or 'casual' in title_lower or 'Sweatpants' in title_lower or 'Sweatpants' in title_lower or 'Hanes' in title_lower or 'sundress' in title_lower or 'matching set' in title_lower  or 'luau' in title_lower or 'hawaiian luau attire' in title_lower or 'hawaiian attire' in title_lower or 'luau attire' in title_lower or 'casual' in title_lower or 'maxi' in title_lower or 'shirt' in title_lower or 'denim' in title_lower or 'sweater' in title_lower:
        return 'Casual'
    elif 'swing' in title_lower or 'rockabilly' in title_lower or 'flapper' in title_lower or 'mod' in title_lower or 'boho' in title_lower:
        return 'Vintage/Retro'
    elif 'tennis' in title_lower or 'golf' in title_lower or 'running' in title_lower or 'gym' in title_lower  or 'sports' in title_lower:
        return 'Sports'
    elif 'princess' in title_lower or 'fairy' in title_lower or 'superhero' in title_lower or 'historical costume' in title_lower or 'character costume' in title_lower:
        return 'Costume'
    elif 'maternity maxi' in title_lower or 'pregnant' in title_lower or 'pregnancy' in title_lower or 'maternity' in title_lower or'maternity wrap' in title_lower or 'maternity bodycon' in title_lower or 'maternity shirt' in title_lower:
        return 'Maternity'
    elif ' casual blazer' in title_lower or 'casual shorts' in title_lower or 'lee' in title_lower or 'hawaiian luau' in title_lower:
        return 'Western Casual'
    elif ' business blazer' in title_lower or 'business' in title_lower or 'blazer' in title_lower or 'formal suit vest' in title_lower or 'formal vest' in title_lower or 'suit vest' in title_lower  or 'formal suit ' in title_lower:
        return 'Western Formal/Business'
    elif ' kaftan' in title_lower or 'thobe' in title_lower  or 'kurti' in title_lower or 'muslim' in title_lower or 'formal vest' in title_lower or 'african' in title_lower or 'thobe' in title_lower or 'tunic' in title_lower:
        return 'Traditional/Ethnic'
   
    elif ' sleep' in title_lower or 'sleeping' in title_lower or 'nighty' in title_lower or 'sleep wear' in title_lower or "sleep wears" in title_lower:
        return 'Night Wear'
    elif 'Slim Fit' in title_lower or 'suit' in title_lower  or 'suits' in title_lower:
        return 'Business'
    elif 'Nike' in title_lower or 'Graphic Fleece Crew' in title_lower or 'Columbia' in title_lower or 'Offshore Short' in title_lower :
        return 'Casual/sporty'
    
    elif ' Waist Trainer' in title_lower or 'Corset Cincher' in title_lower :
        return 'Shape Wear'
    else:
        return 'Other'

# Apply the categorization function to the 'title' column
df['product_type'] = df['title'].apply(categorize_title)

# Print the DataFrame with dress types
print(df)


                                                  title  rating  \
0     dresstells women's formal dresses cocktail tea...      50   
1     dresstells dress with chiffon overlay cape, 20...    1002   
2     blencot womens casual short sleeve boho floral...    1011   
3     hanna nikole women's plus size chiffon ruffle ...    7896   
4     dresstells women's formal dresses cocktail tea...      50   
...                                                 ...     ...   
1445  prinstory men's pajama set sleepwear pjs light...      67   
1446  bulliant mens ratchet belt,leather adjustable ...    5487   
1447  morwenveo men's linen pants casual long pants ...    3556   
1448  hungson mens chinos slim fit stretch flat-fron...    1877   
1449  long skirts for women maxi boho skirt floral p...   10728   

                                                   link  \
0     https://www.amazon.com/sspa/click?ie=UTF8&spc=...   
1     https://www.amazon.com/DRESSTELLS-Dresses-Wedd...   
2     https://www.

In [177]:
df.head()

,title,rating,link,image,size,occasion,color,rate,product_type
0,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,any,3480,Western
1,"dresstells dress with chiffon overlay cape, 20...",1002,https://www.amazon.com/DRESSTELLS-Dresses-Wedd...,https://m.media-amazon.com/images/I/61kTmUZqYW...,"s,m,l,xl",any,any,3728,Western
2,blencot womens casual short sleeve boho floral...,1011,https://www.amazon.com/BLENCOT-Printed-Evening...,https://m.media-amazon.com/images/I/61IVgEOpuB...,"s,m,l,xl",any,any,4060,Western
3,hanna nikole women's plus size chiffon ruffle ...,7896,https://www.amazon.com/Hanna-Nikole-Chiffon-Po...,https://m.media-amazon.com/images/I/51AKCj7rPk...,"s,m,l,xl",any,any,3811,Western
4,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,any,3480,Western


In [178]:
df.isna().sum()

title           0
rating          0
link            0
image           0
size            0
occasion        0
color           0
rate            0
product_type    0
dtype: int64

In [179]:
df['occasion'].replace({'bday':'birthday','wedding,festive occasions':'festival','all':'function'},inplace=True)

In [180]:
df.head(3)

,title,rating,link,image,size,occasion,color,rate,product_type
0,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,any,3480,Western
1,"dresstells dress with chiffon overlay cape, 20...",1002,https://www.amazon.com/DRESSTELLS-Dresses-Wedd...,https://m.media-amazon.com/images/I/61kTmUZqYW...,"s,m,l,xl",any,any,3728,Western
2,blencot womens casual short sleeve boho floral...,1011,https://www.amazon.com/BLENCOT-Printed-Evening...,https://m.media-amazon.com/images/I/61IVgEOpuB...,"s,m,l,xl",any,any,4060,Western


In [181]:
# Creating a new column 'combined' for embedding

df['combined']=df.apply(lambda row: f"{row['title']},{row['color']},{row['occasion']},{row['rate']},{row['product_type']},{row['size']},{row['rating']}",axis=1)

In [182]:
df.isna().sum()

title           0
rating          0
link            0
image           0
size            0
occasion        0
color           0
rate            0
product_type    0
combined        0
dtype: int64

In [183]:
df.product_type.value_counts()

product_type
Western               1124
Traditional            153
Traditional/Ethnic      87
Children                40
Western Casual          18
Other                   11
Casual                   8
Vintage/Retro            5
Ethnic                   3
Sports                   1
Name: count, dtype: int64

In [184]:
df.head(3)

,title,rating,link,image,size,occasion,color,rate,product_type,combined
0,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,any,3480,Western,dresstells women's formal dresses cocktail tea...
1,"dresstells dress with chiffon overlay cape, 20...",1002,https://www.amazon.com/DRESSTELLS-Dresses-Wedd...,https://m.media-amazon.com/images/I/61kTmUZqYW...,"s,m,l,xl",any,any,3728,Western,"dresstells dress with chiffon overlay cape, 20..."
2,blencot womens casual short sleeve boho floral...,1011,https://www.amazon.com/BLENCOT-Printed-Evening...,https://m.media-amazon.com/images/I/61IVgEOpuB...,"s,m,l,xl",any,any,4060,Western,blencot womens casual short sleeve boho floral...


In [185]:
import argparse
import time
import sys
import warnings
import logging

In [186]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
from joblib import dump, load
import pandas as pd

In [187]:

#!pip install tensorflow_text
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
#import tensorflow_hub as hub
!pip install tensorflow-gpu
import tensorflow as tf
from tensorflow import keras
#import tensorflow_text
import sklearn.metrics as metrics

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  Running setup.py install for tensorflow-gpu: started
  Running setup.py install for tensorflow-gpu: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [18 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\ASUS\AppData\Local\Temp\pip-install-ht9ystgw\tensorflow-gpu_b51d22866154470386cfc0f45b248d3a\setup.py", line 37, in <module>
      raise Exception(TF_REMOVAL_WARNING)
  Exception:
  
  The "tensorflow-gpu" package has been removed!
  
  Please install "tensorflow" instead.
  
  Other than the name, the two packages have been identical
  since TensorFlow 2.1, or roughly since Sep 2019. For more
  information, see: pypi.org/project/tensorflow-gpu
  
  
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess-exited-with-error
  
  Running setup.py install for tensorflow-gpu did not 

In [188]:
from sklearn.metrics.pairwise import cosine_similarity

In [189]:

!pip install nltk
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
nltk.download('averaged_perceptron_tagger')


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [190]:
import json
import ast
#to filter the specifications column

In [191]:

!pip install transformers


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [192]:
from transformers import DistilBertModel, DistilBertTokenizer
import torch

import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
     

In [193]:
df.head(3)

,title,rating,link,image,size,occasion,color,rate,product_type,combined
0,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,any,3480,Western,dresstells women's formal dresses cocktail tea...
1,"dresstells dress with chiffon overlay cape, 20...",1002,https://www.amazon.com/DRESSTELLS-Dresses-Wedd...,https://m.media-amazon.com/images/I/61kTmUZqYW...,"s,m,l,xl",any,any,3728,Western,"dresstells dress with chiffon overlay cape, 20..."
2,blencot womens casual short sleeve boho floral...,1011,https://www.amazon.com/BLENCOT-Printed-Evening...,https://m.media-amazon.com/images/I/61IVgEOpuB...,"s,m,l,xl",any,any,4060,Western,blencot womens casual short sleeve boho floral...


In [194]:
df.shape

(1450, 10)

In [195]:
no_rows = 1450
#df = pd.read_csv(DATASET_FILE,nrows=no_rows)
#df = df.drop_duplicates(subset = ["pid"])

df2 = df['title']
# df = df.astype(str)
df = df.dropna()
# df = df.applymap(lambda x: x.lower())
# print(df['description'][:5])
df['title']

0       dresstells women's formal dresses cocktail tea...
1       dresstells dress with chiffon overlay cape, 20...
2       blencot womens casual short sleeve boho floral...
3       hanna nikole women's plus size chiffon ruffle ...
4       dresstells women's formal dresses cocktail tea...
                              ...                        
1445    prinstory men's pajama set sleepwear pjs light...
1446    bulliant mens ratchet belt,leather adjustable ...
1447    morwenveo men's linen pants casual long pants ...
1448    hungson mens chinos slim fit stretch flat-fron...
1449    long skirts for women maxi boho skirt floral p...
Name: title, Length: 1450, dtype: object

In [196]:
# import re
# # Function to extract keys and values from product_specifications column
# def extract_specifications(specifications):
#     pairs = re.findall(r'"key"=>"(.*?)", "value"=>"(.*?)"', specifications)
#     pairs_formatted = [f"{key}: {value}" for key, value in pairs]
#     return ' '.join(pairs_formatted)

# # Apply the function to the product_specifications column
# df['combined'] = df['combined'].apply(extract_specifications)

# print(df['combined'][0])

# # df["product_specifications"][0]

In [197]:

import re

In [199]:

import re
# Function to extract keys and values from product_specifications column
def extract_specifications(specifications):
    pairs = re.findall(r'"key"=>"(.*?)", "value"=>"(.*?)"', specifications)
    pairs_formatted = [f"{key}: {value}" for key, value in pairs]
    return ' '.join(pairs_formatted)


0       dresstells women's formal dresses cocktail tea...
1       dresstells dress with chiffon overlay cape, 20...
2       blencot womens casual short sleeve boho floral...
3       hanna nikole women's plus size chiffon ruffle ...
4       dresstells women's formal dresses cocktail tea...
                              ...                        
1445    prinstory men's pajama set sleepwear pjs light...
1446    bulliant mens ratchet belt,leather adjustable ...
1447    morwenveo men's linen pants casual long pants ...
1448    hungson mens chinos slim fit stretch flat-fron...
1449    long skirts for women maxi boho skirt floral p...
Name: title, Length: 1450, dtype: object

In [158]:
# Apply the function to the product_specifications column
df['title'] = df['title'].apply(extract_specifications)

In [114]:
# def remove_colors(brand):
#     # Define a list of colors to remove
#     colors = [
#     'amber', 'aquamarine', 'aqua', 'beige', 'black', 'blue', 'bronze',
#     'brown', 'burgundy', 'caramel', 'chartreuse', 'charcoal', 'coral',
#     'crimson', 'cyan', 'eggplant', 'emerald', 'forest green', 'fuchsia',
#     'gold', 'golden', 'gray', 'green', 'indigo', 'ivory', 'jade', 'khaki',
#     'lavender', 'lemon', 'lime', 'lilac', 'magenta', 'maroon', 'mint',
#     'mint green', 'moss green', 'mustard', 'navy', 'neon amber',
#     'neon blue', 'neon coral', 'neon crimson', 'neon fuchsia', 'neon gold',
#     # 'neon green', 'neon indigo', 'neon lime', 'neon magenta', 'neon orange',
#     'neon pink', 'neon platinum', 'neon purple', 'neon red', 'neon silver',
#     'neon teal', 'neon turquoise', 'neon violet', 'olive', 'olive green',
#     'orange', 'orchid', 'pearl', 'peach', 'peacock blue', 'peacock green',
#     'periwinkle', 'pink', 'platinum', 'purple', 'red', 'rose', 'rose gold',
#     'ruby', 'ruby red', 'rust', 'salmon', 'salmon pink', 'sapphire',
#     'silver', 'sky blue', 'steel blue', 'tan', 'tangerine', 'taupe', 'teal',
#     'topaz', 'turquoise', 'violet', 'white', 'yellow', 'baby pink', 'baby blue'
#     ]
#     # Create a regular expression pattern to match the colors
#     pattern = r'\b(?:{})\b'.format('|'.join(colors))

#     # Remove the colors from the brand name using regex substitution
#     brand = re.sub(pattern, '', brand, flags=re.IGNORECASE).strip()

#     return brand

In [115]:
# # Apply the remove_colors function to the 'brand' column
# df['title'] = df['title'].apply(remove_colors)


In [200]:
# Combine multiple columns into one
df['combined_text'] = df.apply(lambda row: f"{row['title']} {row['color']} {row['occasion']} {row['size']} {row['rate']} {row['product_type']}", axis=1)
df['combined_text'][0]

"dresstells women's formal dresses cocktail tea party dress 2024 ruffle v-neck modest wedding guest bridesmaid dress any any s,m,l,xl 3480 Western"

In [201]:
df.head(1)

,title,rating,link,image,size,occasion,color,rate,product_type,combined,combined_text
0,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,any,3480,Western,dresstells women's formal dresses cocktail tea...,dresstells women's formal dresses cocktail tea...


In [202]:
df

,title,rating,link,image,size,occasion,color,rate,product_type,combined,combined_text
0,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,any,3480,Western,dresstells women's formal dresses cocktail tea...,dresstells women's formal dresses cocktail tea...
1,"dresstells dress with chiffon overlay cape, 20...",1002,https://www.amazon.com/DRESSTELLS-Dresses-Wedd...,https://m.media-amazon.com/images/I/61kTmUZqYW...,"s,m,l,xl",any,any,3728,Western,"dresstells dress with chiffon overlay cape, 20...","dresstells dress with chiffon overlay cape, 20..."
2,blencot womens casual short sleeve boho floral...,1011,https://www.amazon.com/BLENCOT-Printed-Evening...,https://m.media-amazon.com/images/I/61IVgEOpuB...,"s,m,l,xl",any,any,4060,Western,blencot womens casual short sleeve boho floral...,blencot womens casual short sleeve boho floral...
3,hanna nikole women's plus size chiffon ruffle ...,7896,https://www.amazon.com/Hanna-Nikole-Chiffon-Po...,https://m.media-amazon.com/images/I/51AKCj7rPk...,"s,m,l,xl",any,any,3811,Western,hanna nikole women's plus size chiffon ruffle ...,hanna nikole women's plus size chiffon ruffle ...
4,dresstells women's formal dresses cocktail tea...,50,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/61tcI69u-C...,"s,m,l,xl",any,any,3480,Western,dresstells women's formal dresses cocktail tea...,dresstells women's formal dresses cocktail tea...
...,...,...,...,...,...,...,...,...,...,...,...
1445,prinstory men's pajama set sleepwear pjs light...,67,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/51KJIPWvMn...,"s,m,l,xl",festival,any,1574,Western,prinstory men's pajama set sleepwear pjs light...,prinstory men's pajama set sleepwear pjs light...
1446,"bulliant mens ratchet belt,leather adjustable ...",5487,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/71l9EuW51I...,"s,m,l,xl",festival,any,1077,Western,"bulliant mens ratchet belt,leather adjustable ...","bulliant mens ratchet belt,leather adjustable ..."
1447,morwenveo men's linen pants casual long pants ...,3556,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/51oaRXnxnY...,"s,m,l,xl",festival,any,2237,Casual,morwenveo men's linen pants casual long pants ...,morwenveo men's linen pants casual long pants ...
1448,hungson mens chinos slim fit stretch flat-fron...,1877,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,https://m.media-amazon.com/images/I/41Hnx2ZktV...,"s,m,l,xl",festival,any,2900,Western,hungson mens chinos slim fit stretch flat-fron...,hungson mens chinos slim fit stretch flat-fron...


In [204]:
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('bert-base-nli-mean-tokens')


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 163.1/163.1 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 198.6/198.6 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 10.6/10.6 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 46.2/46.2 MB 4.4 MB/s eta 0:00:00
  Using cached pillow-10.2.0-cp311-cp311-win_amd64.whl (2.6 MB)
     ---------------------------------------- 5.7/5.7 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 3.7 MB/s eta 0:00:00
  Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
     -------------------------------------- 536.2/536.2 kB 6.7 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [205]:
# Get embeddings
sentence_embeddings = bert.encode(df['combined_text'].tolist())

In [223]:
import numpy as np
# Define the file path where you want to save the embeddings
embeddings_path = "C:/Users/ASUS/Downloads/Desktop/Projects/e-commerce-chatbot/embedding"

In [224]:

# def extract_brand(query):
#     query = str(query).lower()  # Convert the query to lowercase for case-insensitive matching
#     brand_names = df['brand'].str.lower()  # Convert the 'brand' column to lowercase for case-insensitive matching

#     for brand in brand_names:
#         if brand in query:
#             return brand

#     return None  # If no brand name is found in the query
# import numpy as np
# embeddings_path = "/content/drive/MyDrive/Colab Notebooks/embeddings2.npy"
# np.save(embeddings_path,sentence_embeddings)

# # userinput = input("enter query") #test
# # print(extract_brand(userinput))
# # print(len(df['brand']))
# # print(len(df))

In [225]:
np.save(embeddings_path,sentence_embeddings)

In [226]:
# userinput = input("enter query") #test
# print(extract_brand(userinput))
# print(len(df['brand']))
# print(len(df))
     

In [237]:
#sentence_embeddings = np.load(embeddings_path) #for future sessions
def extract_product(query):
    query = str(query).lower()  # Convert the query to lowercase for case-insensitive matching
    product_name = df['product_type'].str.lower()  # Convert the 'brand' column to lowercase for case-insensitive matching

    for product in product_name:
        if product in query:
            return product

    return None  # If no brand name is found in the query


In [238]:


#Compute similarity
from sklearn.metrics.pairwise import cosine_similarity

In [242]:
# start = time.time() #start time for recommendation
pd.set_option('display.max_colwidth', None) #to prevent truncation from end of product name

def product_inference(user_input,sentence_embeddings):
  print("Recommendations for \"",user_input,"\" are: ",sep='')
  user_input = re.sub('[^A-Za-z0-9]+', ' ', user_input)
  start=time.time()
  query_brand = extract_product(user_input)
  user_input_embedding = bert.encode([user_input])
  # similarity=cosine_similarity(sentence_embeddings)
  similarities = cosine_similarity(user_input_embedding, sentence_embeddings)
  if query_brand is not None:
    df['product_match_weight'] = df['product'].apply(lambda x: 1.5 if x == query_brand else 1.0)
    df['product_match_weight'] = df['product_match_weight'] * df['title'].apply(lambda x: 1.5  if query_brand in x else 1.0) #now checking for brand name in product_name column too
  else:
    df['product_match_weight'] = 1.0

  similarities_with_brand = similarities*(np.array(df['product_match_weight']))
  num_recommendations = 10
  top_indices = similarities_with_brand.argsort()[0][::-1][:num_recommendations]

  df['product_name_origin']=df2 #to retain the original product names, since the df was preprocessed, which involved removing all symbols etc.
  recommended_products = df.iloc[top_indices]['product_name_origin']
  print(recommended_products)
  end=time.time()-start
  print(end)
input_list = ["white wedding dress under 1000 for men"]

for i in input_list:
  product_inference(i,sentence_embeddings)
  print()


Recommendations for "white wedding dress under 1000 for men" are: 
818                                                                                      yaohuole button down kaftan muslim long gown thobe robe for men white 2xl
812                                                                          men's muslim clothes durable kaftan robe dubai long gown ethnic clothes,1pc soft robe
1163    aditri creation thin dhoti cotton dhoti's full length men's ethnic wear white single piece indian traditional dhoti with 1inch large border new year gifts
1077                                                                                                                    lee uniforms men's long sleeve dress shirt
815                                                                         gafeng mens kaftan thobe long sleeve robe gown casual cotton linen button muslim shirt
856                                                    men's cream cotton dhoti/veshti/mundu set (includes dhoti [3.8m